In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# There are three scenarios for (4,4) currently.
# 1. Stake every Epoch: User claims vested ohm tokens from bond and stakes before following epoch
# 2. Stake daily: User claims vested ohm tokens and stakes once a day
# 3. Stake halfway: User claims vest ohm tokens and stakes half way through the vesting period of 5 days
# In all calculations, gas fees for claim and autostake will be considered
# The simulator function will take a selected scenario as argument

#scenarios = ['stakeEveryEpoch','stakeDaily','stakeHalfway'] # list of scenarios
#selectedScenario = scenarios[0] # Selected scenario


# Variable declarations: Eth and Ohm prices
usdBonded = 46608
initialOhms = 100
#ohmPrice = 466.08
#priceofETH = 2400

# Variable declarations: Protocol
bondDiscount = 6
#bondROI = 5.58
#nextEpochRebase = 0.0040

# Variable declarations: Max network fee
#gwei = 33

# Results data frames: 
# Instantiate an array to hold the results from the simulation
stakingSimulationResults_ROI_df = pd.DataFrame(np.arange(16), columns=['Epochs'])
stakingSimulationResults_ROI_df['Days'] = stakingSimulationResults_ROI_df.Epochs / 3
accumulatedOhmsROI_Staking = []
accumulatedOhms_Staking = []


bondingSimulationResults_ROI_df = pd.DataFrame(np.arange(16), columns=['Epochs'])
bondingSimulationResults_ROI_df['Days'] = bondingSimulationResults_ROI_df.Epochs / 3 
accumulatedOhmsROI_Bonding = []
accumulatedOhms_Bonding = []

# Instantiate a data frame to hold the staking only ohm growth over 5 days
stakingSimulationResults_ohmGrowth_df = pd.DataFrame(np.arange(16), columns=['Epochs'])  
stakingSimulationResults_ohmGrowth_df['Days'] = stakingSimulationResults_ohmGrowth_df.Epochs / 3
#stakingSimulationResults_ohmGrowth_df['Accumulated_Ohms'] = ""

# Instantiate a data frame to hold the staking only ohm growth over 5 days
bondingSimulationResults_ohmGrowth_df = pd.DataFrame(np.arange(16), columns=['Epochs'])  
bondingSimulationResults_ohmGrowth_df['Days'] = bondingSimulationResults_ohmGrowth_df.Epochs / 3
#bondingSimulationResults_ohmGrowth_df['Accumulated_Ohms'] = ""

vestedSimulationResults_df = pd.DataFrame(np.arange(16), columns=['Epochs']) 
vestedSimulationResults_df['Days'] = vestedSimulationResults_df.Epochs / 3
accumulatedOhms_Vested = []

def bondingSimulation(ohmPrice, initialOhms, gwei, priceofETH, bondROI, nextEpochRebase):

# Protocol and ohm calcs:
    discountedOhmPrice = ohmPrice/(1+(bondDiscount/100))
    initOhmValue = initialOhms * ohmPrice
    bondedOhms = initOhmValue/discountedOhmPrice
    bondROI = bondROI/100
#========================================================================================
# Calculate the rebase rate and Current APY (next epoch rebase pulled from hippo data source)

    rewardRate = round(nextEpochRebase/100, 4)
    rebaseConst = 1 + rewardRate  # calculate a constant for use in APY calculation
    currentAPY = (rebaseConst)**(1095)-1  # current APY equation
    currentAPY_P = (currentAPY)*100  # convert to %
#========================================================================================
# Calculate fees
    stakingGasFee = 179123*((gwei*priceofETH)/pow(10,9))
    unstakingGasFee = 89654*((gwei*priceofETH)/(10**9))
    swappingGasFee = 225748*((gwei*priceofETH)/(10**9))+((0.3/100)*initOhmValue)
    claimGasFee = 80209*((gwei*priceofETH)/(10**9))
    bondingGasFee = (258057)*((33*2400)/(10**9))
    # miscFee = 823373 * ((gwei*priceofETH)/(10**9))
    
    claimStakeGasFee = stakingGasFee + claimGasFee
    remainingGasFee = bondingGasFee+unstakingGasFee+swappingGasFee
#========================================================================================= 

# Case selection

#=========================================================================================

# (3,3)
    stakingRate = ((rebaseConst)**15) - 1  # staking reward rate
    stakingRate_P = round(stakingRate * 100, 4) # staking reward rate in percentage
    stakingOhmsGained = ((usdBonded-stakingGasFee)*(stakingRate/usdBonded)-1)
    
    # Calculate ohm roi over 5 day period for comparison
    for x in stakingSimulationResults_ROI_df.Epochs:
        stakingOhmsGrowth_ROI = round(((initOhmValue - stakingGasFee)*((rebaseConst**15)/initOhmValue)-1)*100,3)
        accumulatedOhmsROI_Staking.append(stakingOhmsGrowth_ROI)
    stakingSimulationResults_ROI_df['Staking_ROI_5Days'] = accumulatedOhmsROI_Staking
# ================================================================================

# (4,4)
    bondingRate = (round(bondROI/100, 4))  # bonding reward rate
    bondingRate_P = round(bondingRate * 100, 4) # bonding reward rate in percentage
    #bondingOhmsGained = (usdBonded*bondingRate / discountedOhmPrice)  # ohms gained from bonding 
    
    # Calculate ohm growth over 5 days based on claim/stake frequency. x is the epoch frequency
    for x in bondingSimulationResults_ohmGrowth_df.Epochs:
        bondedOhmsGrowth = round(((bondedOhms/(1+x))*((1.0048**15)-1))/((1.0048**(15/(1+x)))-1),3)
        accumulatedOhms_Bonding.append(bondedOhmsGrowth)
    bondingSimulationResults_ohmGrowth_df['Accumulated_Ohms_Bonding'] = accumulatedOhms_Bonding  

    # Calculate ohm roi over 5 day period for comparison
    for x in bondingSimulationResults_ROI_df.Epochs:
        bondingOhmsGrowth_ROI = round((((bondingSimulationResults_ohmGrowth_df.Accumulated_Ohms_Bonding.iloc[x]*ohmPrice-x*(claimStakeGasFee)-remainingGasFee)/initOhmValue)-1)*100,3)
        #feeOverPeriod = x*(claimStakeGasFee)-remainingGasFee
        accumulatedOhmsROI_Bonding.append(bondingOhmsGrowth_ROI) 
    #bondingOhmsGrowth_ROI = round((((bondingSimulationResults_ohmGrowth_df.Accumulated_Ohms_Bonding*ohmPrice-feeOverPeriod)/initOhmValue)-1)*100,3)    
    bondingSimulationResults_ROI_df['Bonding_ROI_5Days'] = accumulatedOhmsROI_Bonding
# ================================================================================
    
    # Calculate of bonus ohms gained as vested ohms are staked
    #for x in vestedSimulationResults_df.Epochs:
        #vestedOhmsGrowth = ((1+rewardRate)*(1-(1+rewardRate)**x)/(1-(1+rewardRate)))-1
        #accumulatedOhms_Vested.append(vestedOhmsGrowth)
    #vestedSimulationResults_df['Vested_Ohms_5Days'] = accumulatedOhms_Vested
    
# Rates comparison
    rateDiff = bondingRate - stakingRate
    rateDiff_P = rateDiff * 100
    #minBondDiscount = stakingRate_P - vestedBonus_P
# ================================================================================

#ROI and APY
    effectiveRebaseRate = 10**(math.log10(1+bondingRate)/15)-1
    bondingAPY = (1+bondingRate)**(365/5)-1
    bondingAPY_P = bondingAPY * 100
    apyMultiplier = bondingAPY_P/currentAPY_P
#Plot
#    plt.subplot(1,2,1)
#    plt.plot(bondingSimulationResults_ROI_df.Bonding_ROI_5Days,
#            label = 'Bonding ROI over 5 days' )
#    plt.plot(stakingSimulationResults_ROI_df.Staking_ROI_5Days,
#             label = 'Staking ROI over 5 days'
#          )
#    plt.subplot(1,2,2)
#    plt.figure(figsize=(20,8))
#   plt.plot(bondingSimulationResults_ohmGrowth_df.Accumulated_Ohms_Bonding,
#             label = 'Accumulated Ohms over 5 days'
 #           )
#    plt.legend(title = 'Simulated ROI for Staking and Bonding over 5 days')
#    
#    plt.tight_layout()
    return  bondingSimulationResults_ROI_df, bondingSimulationResults_ohmGrowth_df, stakingSimulationResults_ROI_df

In [3]:
bondingSimulation(466.08, 100, 33, 2400, 5.58, 0.48) 

(    Epochs      Days  Bonding_ROI_5Days
 0        0  0.000000              5.603
 1        1  0.333333              7.496
 2        2  0.666667              8.103
 3        3  1.000000              8.386
 4        4  1.333333              8.538
 5        5  1.666667              8.625
 6        6  2.000000              8.675
 7        7  2.333333              8.701
 8        8  2.666667              8.712
 9        9  3.000000              8.711
 10      10  3.333333              8.703
 11      11  3.666667              8.689
 12      12  4.000000              8.670
 13      13  4.333333              8.648
 14      14  4.666667              8.623
 15      15  5.000000              8.595,
     Epochs      Days  Accumulated_Ohms_Bonding
 0        0  0.000000                   106.000
 1        1  0.333333                   107.938
 2        2  0.666667                   108.589
 3        3  1.000000                   108.916
 4        4  1.333333                   109.112
 5        5  1